# Other Variables + RNN on pre-trained Word2vec embedding word vectors 

In [ ]:
down = pd.read_csv('/home/hzhan828/kiva/down.csv')
w2vdict = np.load('/home/hzhan828/kiva/w2vdict.npy').item()

In [118]:
w2vdict = np.load('/Users/zhanhaochen/Desktop/STA160/w2vdict.npy').item()

In [108]:
down = pd.read_csv('/Users/zhanhaochen/Desktop/STA160/down.csv')

In [109]:
test_label = np.load('/Users/zhanhaochen/Desktop/STA160/test_label.npy')

In [147]:
labels = np.reshape(labels,(2,1))

In [160]:
docs = [list(down.description[0:100])] + [list(down.description[-100:-1])]


In [112]:
[down.description[0]] + [down.description[100]]

['Vitaliy lives in the city of Melitopol. He is not married and works as a master of laying paving tiles. <br /><br />Vitaliy purchased a plot of land to develop. This year he would like to lay the fundament and build the walls of his future house. Vitaliy requests a loan of 120,000 hryvnia in order to start the construction. <br /><br />Thanks to the loan, Vitaliy will be able to hire a construction team, which can start working shortly. Vitaliy thanks the Kiva lenders in advance for the help with realizing his dreams - owning a house.',
 'Liduvina attended school through the sixth grade. She has been married for more than 35 years and has 5 children.<br /> <br />Liduvina works in a bookstore—photocopier business that she decided to open with her husband 15 years ago because they saw it as a good income opportunity.<br /> <br />Thanks to her previous Kiva loan she was able to strengthen her business and as seen in the photograph it has remarkably improved. Her last loan is paid off al

In [ ]:
max([len(i) for i in token_doc])

In [191]:
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences


# define documents
docs = down.description
# define class labels
labels = test_label
labels = np.array(labels).reshape(20000,1)
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
# pad documents to a max length of 4 words
max_length = 6050
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')



In [176]:
vocab_size

3504

In [120]:
print(np.mean(down.description_length))

578.293598731818


In [177]:
padded_docs.shape

(200, 6050)

In [128]:
t.word_index.items()

dict_items([('the', 1), ('in', 2), ('a', 3), ('of', 4), ('to', 5), ('vitaliy', 6), ('and', 7), ('r', 8), ('br', 9), ('she', 10), ('her', 11), ('is', 12), ('loan', 13), ('n', 14), ('he', 15), ('married', 16), ('as', 17), ('this', 18), ('his', 19), ('house', 20), ('start', 21), ('construction', 22), ('thanks', 23), ('which', 24), ('for', 25), ('valentine', 26), ('sèmè', 27), ('podji', 28), ('children', 29), ('their', 30), ('school', 31), ('five', 32), ('years', 33), ('ago', 34), ('etc', 35), ('lives', 36), ('city', 37), ('melitopol', 38), ('not', 39), ('works', 40), ('master', 41), ('laying', 42), ('paving', 43), ('tiles', 44), ('purchased', 45), ('plot', 46), ('land', 47), ('develop', 48), ('year', 49), ('would', 50), ('like', 51), ('lay', 52), ('fundament', 53), ('build', 54), ('walls', 55), ('future', 56), ('requests', 57), ('120', 58), ('000', 59), ('hryvnia', 60), ('order', 61), ('will', 62), ('be', 63), ('able', 64), ('hire', 65), ('team', 66), ('can', 67), ('working', 68), ('short

In [129]:
embeddings_index = w2vdict

In [178]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [193]:
embedding_matrix.shape

(3504, 100)

In [140]:
padded_docs

array([[ 6, 36,  2, ...,  0,  0,  0],
       [78, 26,  3, ...,  0,  0,  0]], dtype=int32)

In [203]:
from keras.layers import Input, Concatenate, Dense, Embedding, Flatten, LSTM
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping

In [197]:
embedding_matrix = np.zeros((41291,6050))

## Dropped loan amount because it causes neural network produces worse result.

In [210]:

A1 = Input(shape=(310,),name='A1')

A2 = Dense(512, activation='relu',name='A2')(A1)
A3 = Dense(256, activation='relu',name='A3')(A2)

B1 = Input(shape=(6050,),name='B1')
B2 = Embedding(vocab_size, 300 , weights=[embedding_matrix], input_length=6050, trainable=False, name = 'B2')(B1)
B3 = LSTM(100,dropout=0.2,recurrent_dropout=0.2,name='LSTM')(B2)
B4 = Dense(64, activation='relu',name='B3')(B3)

merged = Concatenate()([A3,B4])
C1 = Dense(320, activation='relu',name='C1')(merged)

C2 = Dense(128, activation='relu',name='C2')(C1)
C3 = Dense(1, activation='softmax',name='C3')(C2)
model = Model(inputs=[A1,B1],outputs = C3)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
B1 (InputLayer)                 (None, 6050)         0                                            
__________________________________________________________________________________________________
A1 (InputLayer)                 (None, 310)          0                                            
__________________________________________________________________________________________________
B2 (Embedding)                  (None, 6050, 100)    4129100     B1[0][0]                         
__________________________________________________________________________________________________
A2 (Dense)                      (None, 512)          159232      A1[0][0]                         
__________________________________________________________________________________________________
LSTM (LSTM

In [ ]:
early_stopping_monitor = EarlyStopping(patience=2)
trainning = model.fit([comb_down,padded_docs], labels, epochs=20, batch_size=None, validation_split=0.1, 
          verbose=True, callbacks = [early_stopping_monitor])

In [ ]:
X_train, X_test, y_train, y_test, pad_train, pad_test= train_test_split(comb_down, test_label,padded_docs, test_size = 0.1, random_state = 0)



In [ ]:
model.evaluate([X_test,pad_test],y_test)